In [ ]:
import urllib.request
import json
import pandas as pd
from datetime import datetime
import seaborn as sns
cm = sns.light_palette("red", as_cmap=True)
#https://www.trilhaseaventuras.com.br/siglas-dos-principais-aeroportos-do-mundo-iata/

#urlOneWay
#https://www.decolar.com/shop/flights-busquets/api/v1/web/search?adults=1&children=0&infants=0&limit=4&site=BR&channel=site&from=POA&to=MIA&departureDate=2020-03-04&groupBy=default&orderBy=total_price_ascending&viewMode=CLUSTER&language=pt_BR&airlineSummary=false&chargesDespegar=false&user=e1861e3a-3357-4a76-861e-3a3357ea76c0&h=38dc1f66dbf4f5c8df105321c3286b5c&flow=SEARCH&di=1-0&clientType=WEB&disambiguationApplied=true&newDisambiguationService=true&initialOrigins=POA&initialDestinations=MIA&pageViewId=62ef8aab-ab53-406c-8429-885702acecbd

In [ ]:
import requests

url = "https://www.pontosmultiplus.com.br/service/facilities/handle-points"

payload = "logado=&select-name=1000&points=1000&action=calculate"
headers = {
  'authority': 'www.pontosmultiplus.com.br',
  'accept': 'application/json, text/javascript, */*; q=0.01',
  'origin': 'https://www.pontosmultiplus.com.br',
  'x-requested-with': 'XMLHttpRequest',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36',
  'dnt': '1',
  'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-mode': 'cors',
  'referer': 'https://www.pontosmultiplus.com.br/facilidades/compradepontos',
  'accept-encoding': 'gzip, deflate, br',
  'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
  'cookie': 'userPrefLanguage=pt_BR; sback_client=573a40fecdbbbb66963e544d; sback_partner=false; sb_days=1549545557254; sback_browser=0-14236400-1548261174b4075e5fdbd390aa38772d39e7c7a352593b045121165093285c48973622c2c1-45877488-170246360, 20525122160-1549545560; sback_customer=$2QUxcVTzd0dOVENUtmd6dlTmp3RjlHVF90bxETQ1oWOad0dWF0QUN3T0hnYBlFVx0UO5BVWTRFVNZTblh2YqRkW2$12; chaordic_browserId=09b01e60-2300-11e9-8ced-6fbc9e419cda; chaordic_anonymousUserId=anon-09b01e60-2300-11e9-8ced-6fbc9e419cda; sback_total_sessions=2; _ducp=eyJfZHVjcCI6ImE4MzY0NWM2LTI3ZWYtNGUzZS1iMzNjLTI3YmY4ZTY4MDMwOCIsIl9kdWNwcHQiOiIifQ==; _fbp=fb.2.1550499169207.1066950068; cto_h2h=B; s_fid=2E4956A0C0C14E48-2CB286BB7EF81637; cto_lwid=01abc4e4-21f3-450f-9f35-57dee229928a; __utmz=196304045.1569964079.10.2.utmcsr=multiplus-emkt|utmccn=20190930_EMAIL_INSTITUCIONAL_BOAS_VINDAS_NOVA_MARCA_BRASIL-20191001|utmcmd=emkt|utmctr=14602|utmcct=cabecalho-ver_extrato_deslogado; s_vnum=1596641437499%26vn%3D2; s_lv=1569964112519; optionExchange=0; origin=[{%22city%22:{%22iataCode%22:%22POA%22%2C%22name%22:%22Porto%20Alegre%22}%2C%22type%22:%22airport%22%2C%22iataCode%22:%22POA%22%2C%22name%22:%22Salgado%20Filho%22%2C%22value%22:%22POA_airport%22%2C%22orderCodeNumber%22:%222%22%2C%22orderCode%22:%22Porto%20Alegre2%22%2C%22label%22:%22Porto%20Alegre%20(POA)%2C%20Salgado%20Filho%20(POA)%2C%20Brasil%22%2C%22position%22:%2200002Porto%20Alegre%20(POA)%2C%20Salgado%20Filho%20(POA)%2C%20Brasil%22}]; destiny=[{%22city%22:{%22iataCode%22:%22FRA%22%2C%22name%22:%22Frankfurt%22}%2C%22type%22:%22airport%22%2C%22iataCode%22:%22FRA%22%2C%22name%22:%22Frankfurt%20Intl.%22%2C%22value%22:%22FRA_airport%22%2C%22orderCodeNumber%22:%222%22%2C%22orderCode%22:%22Frankfurt2%22%2C%22label%22:%22Frankfurt%20(FRA)%2C%20Frankfurt%20Intl.%20(FRA)%2C%20Alemanha%22%2C%22position%22:%2200002Frankfurt%20(FRA)%2C%20Frankfurt%20Intl.%20(FRA)%2C%20Alemanha%22}]; cabinClass=Y; classesSuggestions=[{%22idCabin%22:1%2C%22cabinClass%22:%22Y%22%2C%22cabinName%22:%22Economy%22}%2C{%22idCabin%22:2%2C%22cabinClass%22:%22W%22%2C%22cabinName%22:%22Premium%20Economy%22}%2C{%22idCabin%22:3%2C%22cabinClass%22:%22J%22%2C%22cabinName%22:%22Premium%20Business%22}]; _gcl_au=1.1.278670892.1578924604; _hjid=59ae5b53-f6c8-48b1-bc67-fb8182856ead; chaordic_testGroup=%7B%22experiment%22%3Anull%2C%22group%22%3Anull%2C%22testCode%22%3Anull%2C%22code%22%3Anull%2C%22session%22%3Anull%7D; country_code=br; language_code=pt; __utmc=196304045; _esvan_ref.50060.=; language_country=pt_br; _ga=GA1.3.1171237216.1579530427; _gid=GA1.3.911523691.1579530427; _gaZ=GA1.3.1171237216.1579530427; _gaZ_gid=GA1.3.911523691.1579530427; return=Sat%20Apr%2011%202020%2012:00:00%20GMT-0300%20(Hor%C3%A1rio%20Padr%C3%A3o%20de%20Bras%C3%ADlia); trip=ida_vuelta; departure=Sat%20Apr%2004%202020%2012:00:00%20GMT-0300%20(Hor%C3%A1rio%20Padr%C3%A3o%20de%20Bras%C3%ADlia); SMSESSION=LOGGEDOFF; userIdZ=; __utma=196304045.1744687836.1549545551.1579547569.1579636257.15; analyticsHelper.cd38=ef144e288de8d22700e20cda9fce9ee5ee61b5d25b61bd0dab35f4ddc72e95ce; ATGSESSIONID=yiPNORqQ9P7PZ74G-Syy7CLAjB8uk3Tw0Wc4dHWdUyC7KjCIe4s0u0021-680739279; __zjc7749=4962761565; userTags=%7B%22id%22%3A%22Anonimo%22%2C%22age%22%3A0%2C%22gender%22%3Anull%2C%22email%22%3Anull%2C%22emailHash%22%3Anull%2C%22country%22%3Anull%2C%22city%22%3Anull%2C%22state%22%3Anull%2C%22zipCode%22%3Anull%2C%22typeOfParticipation%22%3Anull%2C%22balance%22%3Anull%2C%22status%22%3A%22deslogado%22%7D; _gac_UA-83192457-1=1.1579696070.CjwKCAiAgqDxBRBTEiwA59eEN-j8nGbsIpfJMIrCCHTfzUi4saF5CmN227pOPsXIuXAOZmOQs_DMSRoCBtMQAvD_BwE; _gcl_aw=GCL.1579696070.CjwKCAiAgqDxBRBTEiwA59eEN-j8nGbsIpfJMIrCCHTfzUi4saF5CmN227pOPsXIuXAOZmOQs_DMSRoCBtMQAvD_BwE; _dc_gtm_UA-83192457-1=1; _gac_UA-83192457-13=1.1579696070.CjwKCAiAgqDxBRBTEiwA59eEN-j8nGbsIpfJMIrCCHTfzUi4saF5CmN227pOPsXIuXAOZmOQs_DMSRoCBtMQAvD_BwE; _dc_gtm_UA-83192457-13=1; __z_a=3200530082274793935727479; JSESSIONID=_hHNOSuko30OZo1X7XyjT4_6rnAXanFcwA7M9PShrPBBjztzhMrIu0021-1010243761; SS_X_JSESSIONID=KoLNOSzOIq0SooUobVecEo7ju0GL-8Y2O_kOVlqjZsm5rKnmkG33u0021-183582721; akavpau_multiplusgeral=1579696676~id=48e1b4d4309a5f9f09664afd46406b0e; __zjc872=4962761577; _gat=1'
}

response = requests.request("POST", url, headers=headers, data = payload)
resultPontos =  response.text.encode('utf8')
resPontos = json.loads(resultPontos.decode('utf-8'))
print(resPontos['data']['total'])
PONTOSMULTIPLUS = resPontos['data']['total']

In [ ]:
dataInicial = '2020-07-03'
dataFinal = '2020-07-19'

idaEvolta=True
#tripType=''

#dataInicial = '2020-04-08'
#dataFinal = '2020-04-22'

#if idaEvolta:
#    tripType = 'roundtrip'
#else:
#    tripType = 'oneway'

specificDate = False

In [ ]:
origens = ['POA','GRU','GIG']
destinos = ['ATL','MIA','MDZ','BRC','LIM','CTG','ADZ','FRA']

#dfDict.append({'de':origem,'para':destino,'Ida': p['departureDate'],'Volta':arr['arrivalDate'],'preco':arr['price']["amount"]})

In [ ]:
resumo = []
dfDict =[]

for origem in origens:
    for destino in destinos:  
        minValue = 999999999
        fraseFinal= ''
        print(origem + ' -> '+ destino)
        urlDecolar = '''https://www.decolar.com/shop/flights-busquets/api/v1/web/calendar-prices/matrix?adults=1&children=0&infants=0&limit=4&site=BR&channel=site&from={origem}&to={destino}&departureDate={dataInicial}&returnDate={dataFinal}&orderBy=total_price_ascending&viewMode=CLUSTER&language=pt_BR&clientType=WEB&initialOrigins={origem}&initialDestinations={destino}&pageViewId=b35e67df-abc9-4308-875f-c3810b3729e4&mustIncludeDates=NA_NA&currency=BRL&breakdownType=TOTAL_FARE_ONLY'''.format(dataInicial=dataInicial,dataFinal=dataFinal,origem=origem,destino=destino)
        #print(urlDecolar)
        with urllib.request.urlopen(urlDecolar) as url:
            s = url.read()
            data = json.loads(s.decode('utf-8'))
            #print(data)
            for p in data['departures']:
                for arr in p['arrivals']:
                    if 'price' in arr:
                        dfDict.append({'DataPesquisa':datetime.now().strftime("%d/%m/%Y %H:%M:%S"),'de':origem,'para':destino,'Ida': p['departureDate'],'Volta':arr['arrivalDate'],'preco':arr['price']["amount"]})
                        if specificDate:
                            if p['departureDate'] == dataInicial and arr['arrivalDate'] == dataFinal:                                                                
                                    if minValue > arr['price']["amount"]:
                                        minValue = arr['price']["amount"]
                                        fraseFinal = 'Voo mais barato '+origem + ' -> '+ destino+' de:' + p['departureDate'], ' ate ',arr['arrivalDate'],'- valor: ' + str(arr['price']["amount"])                                
                                        resumo.append(fraseFinal)
                                    print('de:' + p['departureDate'], ' ate ',arr['arrivalDate'],'- valor: ' + str(arr['price']["amount"]))                        
                        else:                        
                                if minValue > arr['price']["amount"]:
                                    minValue = arr['price']["amount"]
                                    fraseFinal = 'Voo mais barato '+origem + ' -> '+ destino+' de:' + p['departureDate'], ' ate ',arr['arrivalDate'],'- valor: ' + str(arr['price']["amount"])                                
                                    resumo.append(fraseFinal)
                                print('de:' + p['departureDate'], ' ate ',arr['arrivalDate'],'- valor: ' + str(arr['price']["amount"]))                        
            print('')
            print(fraseFinal)
            print(minValue)
            print('')
                        
for r in resumo:
    print(r)
                    

In [ ]:
df = pd.DataFrame.from_dict(dfDict)

if specificDate:
    df = df[df['Ida']==dataInicial]
    df = df[df['Volta']==dataFinal]
display(df.describe())
df.sort_values(by='preco',ascending=True).head(5).style.background_gradient(cmap='OrRd')

In [ ]:
with open('historicoPesquisaPrecos.csv', 'a') as f:
    df.to_csv(f, mode='a',header=f.tell()==0)
    

In [ ]:
dfGrafico = pd.read_csv("historicoPesquisaPrecos.csv") 


In [ ]:
dfGrafico = dfGrafico[dfGrafico['Ida']>='2020-07-03']
dfGrafico = dfGrafico[dfGrafico['Ida']<='2020-07-07']
dfGrafico = dfGrafico[dfGrafico['Volta']>='2020-07-17']
dfGrafico = dfGrafico[dfGrafico['Volta']<='2020-07-20']

In [ ]:
dfGrafico['DataPesquisa'] = dfGrafico['DataPesquisa'].apply(lambda x:x[0:13])
dfGrafico['DataPesquisaDATA']=dfGrafico['DataPesquisa'].apply(lambda x:pd.to_datetime(x[0:10]))
dfGrafico['Dias'] = dfGrafico.apply(lambda x: int(str(pd.to_datetime(x['Volta'])- pd.to_datetime(x['Ida']))[0:2]),axis=1)

In [ ]:
dfGrafico['OrigemDestino'] = dfGrafico.apply(lambda x: x['de'] + x['para'],axis=1)
dfGrafico['EspecificoIda'] = dfGrafico.apply(lambda x: x['de'] + x['para']+'-'+x['Ida'],axis=1)
dfGrafico['EspecificoVolta'] = dfGrafico.apply(lambda x: x['de'] + x['para']+'-'+x['Volta'],axis=1)
dfGrafico['EspecificoTodos'] = dfGrafico.apply(lambda x: x['de'] + x['para']+'-'+x['Ida']+'-'+x['Volta'],axis=1)
display(dfGrafico)

In [ ]:
#dfGraficoPOA_ATL = dfGrafico.query('de == "POA" & para == "ATL"')
#dfGraficoPOA_MIA = dfGrafico.query('de == "POA" & para == "MIA"')
#dfGraficoGRU_MIA = dfGrafico.query('de == "GRU" & para == "MIA"')
#dfGraficoGRU_ATL = dfGrafico.query('de == "GRU" & para == "ATL"')
#dfGraficoGRU_MDZ = dfGrafico.query('de == "GRU" & para == "MDZ"')
#dfGraficoPOA_MDZ = dfGrafico.query('de == "POA" & para == "MDZ"')

#datasets = [dfGrafico,dfGraficoPOA_ATL,dfGraficoPOA_MIA,dfGraficoGRU_MIA,dfGraficoGRU_ATL,dfGraficoGRU_MDZ,dfGraficoPOA_MDZ]

#print(dfGraficoPOA_ATL['Ida'].count())
#print(dfGraficoPOA_MIA['Ida'].count())
#print(dfGraficoGRU_MIA['Ida'].count())
#print(dfGraficoGRU_ATL['Ida'].count())
#print(dfGraficoGRU_MDZ['Ida'].count())
#print(dfGraficoPOA_MDZ['Ida'].count())

In [ ]:
#import plotly.express as px

#for graph in datasets: 
#    #graph = graph.query('Ida =="2020-07-05" & Volta =="2020-07-20"')
#    graph = graph.query('de =="POA" & Dias >=14 & Dias <=17')# | de =="GRU"')
#    fig = px.line(graph.drop_duplicates(), x="DataPesquisa", y="preco", color="EspecificoTodos",hover_data=['de','para','Ida', 'Volta','preco'])
#    fig.show()

In [ ]:
import pandas_profiling
print(dfGraficoPOA_MIA.columns)

In [ ]:
#pandasDf=dfGraficoPOA_MIA[['Ida', 'Volta', 'de', 'para', 'preco','DataPesquisaDATA', 'Dias']]
#display(pandasDf.head(3))
#pandas_profiling.ProfileReport(pandasDf)


In [ ]:
dfPivot = dfGrafico.query('de == "POA" or de=="GRU"')
#display(dfPivot.head(3))

In [ ]:
dfPivot = pd.pivot_table(dfPivot,values='preco',index=['de','para','Dias','Ida'],columns='DataPesquisa')

## Maiores valores da serie historica

In [ ]:
#display(dfPivot)
#dfPivot.style.apply(highlight_max)

## Menores valores da serie historica

In [ ]:
#dfPivot.style.apply(highlight_min)

In [ ]:
#dfLastSearch = dfGrafico.query('de == "POA" or de=="GRU"')
#print(dfLastSearch.groupby(['de','para']).count())
#dfLastSearch = dfLastSearch[dfLastSearch['DataPesquisaDATA']>='21/01/2020']
#dfLastSearchPivot = pd.pivot_table(dfLastSearch,values='preco',index=['de','para','Dias','Ida','Volta'],columns='DataPesquisa')

In [ ]:
#dfLastSearchPivot.style.apply(highlight_min)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

#df = pd.DataFrame([[3,2,10,4],[20,1,3,2],[5,4,6,1]])


In [ ]:
#dfLastSearchPivot.fillna(0,inplace=True)
#dfLastSearchPivot.query('para == "MIA"').style.background_gradient(cmap='OrRd')
#display(dfLastSearchPivot.style.background_gradient(cmap='OrRd'))
#print(dfLastSearchPivot.groupby(['de','para']).count())
#dfLastSearchPivot.style.apply(background_gradient,cmap='OrRd',m=dfLastSearchPivot.min().min(),M=dfLastSearchPivot.max().max(),low=0,high=7000)

In [ ]:
urlPontoLatam = 'https://bff.latam.com/ws/proxy/booking-webapp-bff/v1/public/redemption/recommendations/outbound?departure={dataInicial}&origin={origem}&destination={destino}&cabin=Y&country=BR&language=PT&home=pt_br&return={dataFinal}&adult=1&tierCode=LTAM&tierType=low'
origensPontos = ['POA','GRU','GIG']
destinosPontos = ['ATL','MIA','MDZ','BRC','LIM','CTG','ADZ','FRA']
dataPesquisa = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
dfPontosListIda =[]
dfPontosListVolta =[]
meuSaldoAtual = 22000

for origem in origensPontos:
    for destino in destinosPontos:  
        minValue = 999999999
        fraseFinal= ''
        print(origem + ' -> '+ destino)
        urlPontos = urlPontoLatam.format(dataInicial=dataInicial,dataFinal=dataFinal,origem=origem,destino=destino)
        #print(urlDecolar)
        with urllib.request.urlopen(urlPontos) as url:
            s = url.read()
            data = json.loads(s.decode('utf-8'))
            try:
                    
                for flight in data['data']:                  
                    for cabins in flight['flights']:                    
                        paradas = cabins['stops']
                        dataChegada=cabins['arrival']['date']
                        horaChegada = cabins['arrival']['time']['hours']
                        minutoChegada = cabins['arrival']['time']['minutes']
                        overnight = cabins['arrival']['overnights']
                        #partida
                        dataPartida=cabins['departure']['date']
                        horaPartida = cabins['departure']['time']['hours']
                        minutoPartida = cabins['departure']['time']['minutes']                                                            
                        for price in cabins['cabins']:
                            dfPontosListIda.append({'DataPesquisa':dataPesquisa,'De':origem,'Para':destino,'PartidaData':dataPartida,'PartidaHora':horaPartida,'PartidaMinuto':minutoPartida,'ChegadaData':dataChegada,'ChegadaHora':horaChegada,'ChegadaMinuto':minutoChegada,'overnight':overnight,'Paradas':paradas,'pontos':price['displayPrice'],'preco':(PONTOSMULTIPLUS *price['displayPrice'])/1000,'precoMenosSaldo':(PONTOSMULTIPLUS *(price['displayPrice']-meuSaldoAtual))/1000})                                                
                            dfPontosIda = pd.DataFrame.from_dict(dfPontosListIda)
            except:
                print('erro')
        
        print(destino + ' -> '+ origem)
        urlPontos = urlPontoLatam.format(dataInicial=dataFinal,dataFinal=dataFinal,origem=destino,destino=origem)
        with urllib.request.urlopen(urlPontos) as url:
            s = url.read()
            data = json.loads(s.decode('utf-8'))
            try:
                    
                for flight in data['data']:                  
                    for cabins in flight['flights']:                    
                        paradas = cabins['stops']
                        dataChegada=cabins['arrival']['date']
                        horaChegada = cabins['arrival']['time']['hours']
                        minutoChegada = cabins['arrival']['time']['minutes']
                        overnight = cabins['arrival']['overnights']
                        #partida
                        dataPartida=cabins['departure']['date']
                        horaPartida = cabins['departure']['time']['hours']
                        minutoPartida = cabins['departure']['time']['minutes']                                                            
                        for price in cabins['cabins']:
                            dfPontosListVolta.append({'DataPesquisa':dataPesquisa,'De':destino,'Para':origem,'PartidaData':dataPartida,'PartidaHora':horaPartida,'PartidaMinuto':minutoPartida,'ChegadaData':dataChegada,'ChegadaHora':horaChegada,'ChegadaMinuto':minutoChegada,'overnight':overnight,'Paradas':paradas,'pontos':price['displayPrice'],'valorPontos':PONTOSMULTIPLUS,'preco':(PONTOSMULTIPLUS *price['displayPrice'])/1000,'precoMenosSaldo':(PONTOSMULTIPLUS *(price['displayPrice']-meuSaldoAtual))/1000})                                                
                            dfPontosVolta = pd.DataFrame.from_dict(dfPontosListVolta)   
            except:
                print('erro')

In [ ]:
with open('historicoPesquisaPontosIda.csv', 'a') as f:
    dfPontosIda.to_csv(f, mode='a',header=f.tell()==0)

with open('historicoPesquisaPontosVolta.csv', 'a') as f:
    dfPontosVolta.to_csv(f, mode='a',header=f.tell()==0)

In [ ]:
#dfLoadPontosIda = pd.read_csv("historicoPesquisaPontosIda.csv") 
#dfLoadPontosVolta = pd.read_csv("historicoPesquisaPontosVolta.csv") 

In [ ]:
#dfPontosC = dfLoadPontosVolta[['DataPesquisa','De','Para','PartidaData','PartidaHora', 'PartidaMinuto','ChegadaData', 'ChegadaHora', 'ChegadaMinuto','Paradas','overnight', 'pontos', 'preco','precoMenosSaldo']]
#display(dfPontosC.sort_values(by='preco',ascending=True).style.background_gradient(cmap='OrRd'))


In [ ]:
uriPontos = 'https://www.pontosmultiplus.com.br/service/facilities/handle-points'

In [ ]:
#dfT = dfLastSearch
#dfTeste = dfT[dfT['DataPesquisaDATA']=='24/01/2020']
#dfTeste = pd.pivot_table(dfLastSearch,values='preco',index=['de','para','Ida'],columns='Volta')
#dfTeste.fillna(0,inplace=True)

In [ ]:
#display(dfTeste.style.background_gradient(cmap='OrRd'))

In [ ]:
aa

In [ ]:
#POSTMAN ONE WAY 

In [ ]:
import requests

dataInicial = '2020-04-08'
dataFinal = '2020-04-22'

origens = ['POA','GRU','GIG','BSB','FOR']
destinos = ['ATL','MIA']

url = "https://www.decolar.com/shop/flights-busquets/api/v1/web/search"

for origem in origens:
    for destino in destinos:        
        querystring = {"adults":"1","limit":"4","site":"BR","channel":"site","from":"{origem}".format(origem=origem),"to":"{destino}".format(destino=destino),"departureDate":"2020-03-04","orderBy":"total_price_ascending","viewMode":"CLUSTER","language":"pt_BR","h":"38dc1f66dbf4f5c8df105321c3286b5c","flow":"SEARCH","clientType":"WEB","initialOrigins":"{origem}".format(origem=origem),"initialDestinations":"{destino}".format(destino=destino)}
    
        headers = {
            'Connection': "keep-alive",
            'DNT': "1",
            'X-UOW': "results-13-1579106681089",
            'X-RequestId': "xzTTJ6fDfw",
            'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
            'Accept': "application/json, text/javascript, */*; q=0.01",
            'X-Requested-With': "XMLHttpRequest",
            'XDESP-REFERRER': "https://www.decolar.com/shop/flights/search/oneway/{origem}/{destino}/2020-03-04/2/0/0/NA/NA/NA/NA/?from=SB&di=2-0".format(origem=origem,destino=destino),
            'Sec-Fetch-Site': "same-origin",
            'Sec-Fetch-Mode': "cors",
            'Referer': "https://www.decolar.com/shop/flights/search/oneway/{origem}/{destino}/2020-03-04/1/0/0/NA/NA/NA/NA/?from=SB&di=1-0".format(origem=origem,destino=destino),
            'Accept-Encoding': "gzip, deflate, br",
            'Accept-Language': "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
            'Cookie': 'trackerid=e1861e3a-3357-4a76-861e-3a3357ea76c0; xdesp-rand-usr=292; xdsid=C632CEAAF251AE2A72F165ECA9A4A2CA; xduid=1727A02D2FAA249C654A094113369154; _ga=GA1.2.772144563.1579011917; _gid=GA1.2.317154519.1579011917; trackeame_cookie=%7B%22id%22%3A%22UPA_e1861e3a-3357-4a76-861e-3a3357ea76c0%22%2C%22version%22%3A%225.0%22%2C%22upa_id%22%3A%22e1861e3a-3357-4a76-861e-3a3357ea76c0%22%2C%22creation_date%22%3A%222020-01-14T14%3A25%3A17Z%22%7D; __ssid=41de76d348be0e334af8e657f6801b8; _gcl_au=1.1.1367791908.1579011932; _fbp=fb.1.1579011933564.1470255143; __gads=ID=9139db3a836078f5:T=1579011933:S=ALNI_MawboBo55i9nPvoDvzaF396HudEKg; abzTestingId="{\"flightsFisherAB\":90,\"pkgImbatibleBrand_ctrl\":76,\"s_flights_s_violet_sbox_v1\":21,\"upsellingConfig\":58,\"twoOneWayForceMX\":0,\"filterLandingFlights\":41,\"s_loyalty_v2_ctrl\":5,\"s_flights_l_violet_sbox_v1\":0,\"s_flights_l_loyalty_v2\":58,\"mostProfitablePromotion\":0,\"despechecks\":72,\"s_loyalty_v2_review\":33,\"platform\":55,\"selected_radio_button\":0,\"fisher_2ow\":0,\"loyalty_non_adherents\":63,\"paymentMethod\":55,\"shifuMobileProductLabels\":0,\"obFee\":40,\"twoOneWay\":0,\"s_violet_sbox_v1\":17,\"s_flights_s_loyalty_v2\":14,\"flights_loyalty_non_adherents\":63,\"pkgImbatibleBrand-ctrl\":60,\"crossBorderTicketing\":0}; chktkn=ask3r5kj6ed0ksqrs7eio4cebk; searchId=243920d8-49cc-4271-972a-60d05221ef20; _gat_UA-36944350-2=1,trackerid=e1861e3a-3357-4a76-861e-3a3357ea76c0; xdesp-rand-usr=292; xdsid=C632CEAAF251AE2A72F165ECA9A4A2CA; xduid=1727A02D2FAA249C654A094113369154; _ga=GA1.2.772144563.1579011917; _gid=GA1.2.317154519.1579011917; trackeame_cookie=%7B%22id%22%3A%22UPA_e1861e3a-3357-4a76-861e-3a3357ea76c0%22%2C%22version%22%3A%225.0%22%2C%22upa_id%22%3A%22e1861e3a-3357-4a76-861e-3a3357ea76c0%22%2C%22creation_date%22%3A%222020-01-14T14%3A25%3A17Z%22%7D; __ssid=41de76d348be0e334af8e657f6801b8; _gcl_au=1.1.1367791908.1579011932; _fbp=fb.1.1579011933564.1470255143; __gads=ID=9139db3a836078f5:T=1579011933:S=ALNI_MawboBo55i9nPvoDvzaF396HudEKg; abzTestingId="{\"flightsFisherAB\":90,\"pkgImbatibleBrand_ctrl\":76,\"s_flights_s_violet_sbox_v1\":21,\"upsellingConfig\":58,\"twoOneWayForceMX\":0,\"filterLandingFlights\":41,\"s_loyalty_v2_ctrl\":5,\"s_flights_l_violet_sbox_v1\":0,\"s_flights_l_loyalty_v2\":58,\"mostProfitablePromotion\":0,\"despechecks\":72,\"s_loyalty_v2_review\":33,\"platform\":55,\"selected_radio_button\":0,\"fisher_2ow\":0,\"loyalty_non_adherents\":63,\"paymentMethod\":55,\"shifuMobileProductLabels\":0,\"obFee\":40,\"twoOneWay\":0,\"s_violet_sbox_v1\":17,\"s_flights_s_loyalty_v2\":14,\"flights_loyalty_non_adherents\":63,\"pkgImbatibleBrand-ctrl\":60,\"crossBorderTicketing\":0}"; chktkn=ask3r5kj6ed0ksqrs7eio4cebk; searchId=243920d8-49cc-4271-972a-60d05221ef20; _gat_UA-36944350-2=1; xdsid=DCF9EDC0035E07BEDBFEE30E55F725C5; xduid=55D857BEFC5E27A8B84A7407D4A86B38; xdesp-rand-usr=292; abzTestingId="{\"flightsFisherAB\":90,\"pkgImbatibleBrand_ctrl\":76,\"s_flights_s_violet_sbox_v1\":21,\"upsellingConfig\":58,\"twoOneWayForceMX\":0,\"filterLandingFlights\":41,\"s_loyalty_v2_ctrl\":5,\"s_flights_l_violet_sbox_v1\":0,\"s_flights_l_loyalty_v2\":58,\"mostProfitablePromotion\":0,\"despechecks\":72,\"s_loyalty_v2_review\":33,\"platform\":55,\"selected_radio_button\":0,\"fisher_2ow\":0,\"loyalty_non_adherents\":63,\"paymentMethod\":55,\"shifuMobileProductLabels\":0,\"obFee\":40,\"twoOneWay\":0,\"s_violet_sbox_v1\":17,\"s_flights_s_loyalty_v2\":14,\"flights_loyalty_non_adherents\":63,\"pkgImbatibleBrand-ctrl\":60,\"crossBorderTicketing\":0}',
            'Cache-Control': "no-cache",
            'Postman-Token': "4c6c6b9f-ed0a-477f-a787-c8cde039475b,4e35a9da-93ed-4602-825a-283f619d543b",
            'Host': "www.decolar.com",
            'cache-control': "no-cache"
            }

        response = requests.request("GET", url, headers=headers, params=querystring)
        dataOneWay = json.loads(response.text)
        print(origem, '->' , destino)
        print(querystring)
        print(dataOneWay)
        if 'clusters' in dataOneWay:
            for i in dataOneWay['clusters']:                
                print(i['priceDetail']['mainFare']['amount'])


